In [1]:
from pydemic.all import *
from pydemic import fitting as fit
from functools import lru_cache

In [2]:
mundi.region('BR').pydemic.epidemic_curve()

,cases,deaths
date,,
2020-03-31,628,0
2020-04-01,1758,0
2020-04-02,2963,0
2020-04-03,4117,0
2020-04-04,5267,0
...,...,...
2020-08-02,2700672,93600
2020-08-03,2724649,94245
2020-08-04,2773158,95339


In [3]:
        
        
def suspect(model, dates):
    return model['cases:dates'].loc[dates] / 0.13


In [4]:
states = mundi.regions(country_code='BR', type='city').index
report = Report.from_options(SEAIR, region=states)
report.init_cases().init_R0().run(60)

NameError: name 'Report' is not defined

In [129]:
def to_column(col):
    if callable(col):
        return col
    
    def fn(model, dates):
        return model[col + ':dates'].loc[dates]
        
    return fn

def col_name(col):
    if isinstance(col, str):
        return col
    return col.__name__

def get_times(self, columns, times=(7, 15, 30, 60), prepend=None, append=False, dtype=None):
    locs = [t - self._niter - 1 for t in times]
    col_names = [*map(col_name, columns)]
    columns = [*map(to_column,columns)]
    rows = []
    n_times = len(times)
    n_cols = len(columns)
    
    for m in self._models:
        m = m.clinical()
        dates = m.dates[locs]
        row = [None] * (n_times * n_cols)
        rows.append(row)
        for i, col in enumerate(columns):
            values = col(m, dates)
            row[i::n_cols] = values
    
    index = [m.region.id for m in self._models]
    col_tuples = ((x, y) for x in times for y in col_names)
    col_index = pd.MultiIndex.from_tuples(col_tuples)
    data = pd.DataFrame(rows, index=index, columns=col_index)
    if dtype:
        data = data.astype(dtype)
    
    prefix = pd.DataFrame([m.R0 for m in self._models], index=index, columns=[('info', 'R0')])
    data = pd.concat([prefix, data], axis=1)
    data.columns = pd.MultiIndex.from_tuples(data.columns)
    return data

get_times(report, ['cases', 'deaths', 'severe', 'critical', suspect], dtype=int).to_excel('br.xlsx')

In [130]:
!pwd

/home/chips/git/corona/pydemic/notebooks


In [58]:
models = {
    r: region(r)[['cases:dates', 'deaths:dates', 'severe:dates', 'critical:dates']].loc['2020-07-29':].T
    for r in regions_sp()
}

In [56]:
len(models)

663

In [60]:
data = pd.concat([df for df in models.values()], keys=models.keys())

In [61]:
data.to_excel('sp.xlsx')